In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.artist as art
import seaborn as sns
#seaborn styling
sns.set(style="whitegrid")
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode()

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#acquire and clean data further
df = pd.read_csv("main_df.csv",low_memory=False)
df.category = np.where(df.item_group=="Court Time", "Admin",df.category)
df = df[df.category!="Program"]

#convert to timestamp
dft = df.copy()
dft.timestamp = pd.to_datetime(dft.timestamp)
dft.set_index("timestamp", inplace=True)

dft["day"] = dft.index.strftime("%w %a")
dft["year"] = dft.index.strftime("%Y")
dft["month"] = dft.index.strftime("%m %B")

In [ ]:
#items
admin_items = list(df[df.category=="Admin"].item_group.unique())
bar_items = list(df[df.category=="Bar"].item_group.unique())
tennis_items = list(df[df.category=="Tennis"].item_group.unique())
fitness_items = list(df[df.category=="Fitness"].item_group.unique())
ps_items = list(df[df.category=="Pro Shop"].item_group.unique())
event_items = list(df[df.category=="Event"].item_group.unique())

## How to properly evaluate the data

1. Data was generated exclusively from Club Automation POS system. Because JNCC uses other POS system, the insights should not be assumed to mirror JNCC's revenue. However because the size of data is relatively large, the **trends and patterns likely reflect actual movements**.
  
  
2. Product category **Program** was excluded because payments are moved to RegFox, which is another POS system. Program category includes historical sales from the following:
     - Swim Lessons
     - Camp
     - Swim Camp
     - Kid Fit
     - Teen Performance Camp


3. The product categories and items included in the data are deliberately chosen to help forecast items that are currently offered by JNCC. An example of items removed from the analysis is Soccer club sales, which was previously in the Club Automation POS but had been moved to another POS system, separate from JNCC. Below is the list of item groups per product category.


In [ ]:
pd.DataFrame(data=[admin_items,bar_items,tennis_items,fitness_items,ps_items,event_items],index=["Admin","Bar","Tennis","Fitness","Pro Shop","Events"]).fillna("").T

## Outline

- I. Product Categories  
 - A. Trends & Seasonality
    1. Revenue and engagement as distinct KPIs
    2. Product categories and their corresponding trend
    3. Seasonalities in product categories
  - B. Forecasts
    1. Model Performance
    2. How to use 2020 Forecast Report

---

## A. Trends & Seasonality

### 1. Summary of Year-end Revenue

In [ ]:
yearend_revenue = round(dft.pivot_table(values="total",columns="category",index="year",aggfunc=sum,margins=True),2)
yearend_revenue

### 2. Summary of Year-end Engagement

In [ ]:
yearend_engagement = round(dft.pivot_table(values="trans_id",columns="category",index="year",aggfunc="count",margins=True),2)
yearend_engagement

In [ ]:
to = dft[dft.category == "Tennis"]

In [ ]:
to.shape

In [ ]:
to_summary = to.groupby("year").agg({"total":"sum","trans_id":"count"}).rename(columns={"total":"revenue","trans_id":"engagement"})

In [ ]:
to_summary

In [ ]:
rev_eng = dft.groupby(["category"]).resample("6M").agg({"total":sum,"trans_id":"count"}).reset_index().rename(columns={"total":"dollar_value","trans_id":"frequency"})

In [ ]:
rev_eng["dollar_value_scaled"] = MinMaxScaler().fit_transform(rev_eng[["dollar_value","frequency"]])[:,0]
rev_eng["frequency_scaled"] = MinMaxScaler().fit_transform(rev_eng[["dollar_value","frequency"]])[:,1]

In [ ]:
rev_eng.timestamp = rev_eng.timestamp.dt.strftime("%b-%y")
rev_eng.set_index("timestamp",inplace=True)

In [ ]:
def plot_axis(ax,cat,df,s1,s2,c1="steelblue",c2="tomato",xmax=11,lw=1):
    ax.plot(df[s1][df["category"]==cat],color=c1,lw=lw,label="revenue")
    ax.plot(df[s2][df["category"]==cat],color=c2,lw=lw,label="engagement")
    ax.get_yaxis().set_visible(False)
    ax.set_title(cat)
#     ax.hlines(y=0,xmin=0,xmax=xmax,ls=":",color="black")

In [ ]:
fig = plt.figure(figsize=(16,10))
fig.suptitle("What Do Trends in Revenue and Engagement Look Like?")

ax1 = fig.add_subplot(321,frameon=False)
ax2 = fig.add_subplot(322,frameon=False)
ax3 = fig.add_subplot(323,frameon=False)
ax4 = fig.add_subplot(324,frameon=False)
ax5 = fig.add_subplot(325,frameon=False)
ax6 = fig.add_subplot(326,frameon=False)

plt.subplots_adjust(left=None, bottom=None, right=None, top=0.9, wspace=None, hspace=None)

plot_axis(ax1,"Admin",rev_eng,"dollar_value_scaled","frequency_scaled",xmax=10)
sub1 = plot_axis(ax2,"Bar",rev_eng,"dollar_value_scaled","frequency_scaled",xmax=10,lw=3)
plot_axis(ax3,"Tennis",rev_eng,"dollar_value_scaled","frequency_scaled",xmax=10)
plot_axis(ax4,"Fitness",rev_eng,"dollar_value_scaled","frequency_scaled",xmax=10)
plot_axis(ax5,"Event",rev_eng,"dollar_value_scaled","frequency_scaled",xmax=10)
sub3 = plot_axis(ax6,"Pro Shop",rev_eng,"dollar_value_scaled","frequency_scaled",xmax=10,lw=3)

fig.legend([sub1],labels=["revenue","engagement"],loc='upper center', ncol=2, borderaxespad=2.5, frameon=False)
plt.subplots_adjust(hspace=0.4)
plt.show()

What: **Scaled representation of Revenue and Engagement**
 - If the engagement and revenue are proportionate to each other, the red line (engagement) sits just below the blue line (revenue) and follows roughly the same movement.  
  
So What: **Revenue and Engagement in Bar is oppositely proportionate, and in Pro Shop is disroportionate**.
 - Bar: While the engagement touchpoints are high (customers buy frequently), the items are priced relatively lower. 
 - Pro Shop: The engagement touchpoints are not generating desired revenue. This pattern likely happens when revenue is highly affected by an abnormally-high or abnormally low-priced item. In other words the variance of prices is big. Maybe only certain Pro Shop goods are driving revenue, but everything else are not revenue-generating.
 > Looking at the Pro Shop items further, we see the following:  
     - Racquets are generating high revenue but at a lower frequency
     - Shoe and apparel are adequately priced and follow the ~1:2 ratio (engagement:revenue)
     - The rest are low priced items that generating visible (but low) revenue because of the amount of items purchased.

In [ ]:
ps = df[df.category=="Pro Shop"]
ps = ps.groupby("item_group").agg({"total":sum,"trans_id":"count"}).rename(columns={"total":"revenue","trans_id":"no_transactions"}).reset_index()

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,5))

lab1 = ax1.bar(x=ps.item_group,height=ps.no_transactions,color="lightgray",label="number of transactions")
ax1.grid(False)
ax1.set_ylabel("No. of Items Sold")

ax2 = ax1.twinx()
lab2 = ax2.plot(ps.revenue,color="tomato",marker="s",label="revenue")
ax2.set_ylabel('', color='tomato')
ax2.tick_params('y', colors='tomato')
ax2.grid(False)
ax2.set_ylabel("Revenue in $")


plt.suptitle("How Does Engagement and Revenue Compare in Various Pro Shop Items?")
plt.legend((lab1,lab2),("y","z"),loc='upper center', ncol=1, borderaxespad=0, facecolor="inherit", frameon=False)
plt.show()

### 3. Decomposing Trends

In [ ]:
fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

categories = {"Admin":ax1,"Bar":ax2,"Tennis":ax3,"Fitness":ax4,"Event":ax5,"Pro Shop":ax6}

for c,axis in categories.items():
    decomposition_rev = sm.tsa.seasonal_decompose(dft[dft["category"]==c]["total"].resample('Q').sum(), model='additive')
    trend_rev = decomposition_rev.trend
    trend_rev.plot(ax=axis, title=f"{c}",color="#328B9B")
    if (c == "Event") or (c == "Pro Shop"):
        axis.get_xaxis().set_visible(True)
        axis.get_yaxis().set_visible(False)
        axis.grid(False)
    else:
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        
plt.subplots_adjust(hspace=0.4)
# plt.savefig('graph/trend_rev.png', transparent=True)

In [ ]:
fig = plt.figure(figsize=(16,7))

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

categories = {"Admin":ax1,"Bar":ax2,"Tennis":ax3,"Fitness":ax4,"Event":ax5,"Pro Shop":ax6}

for c,axis in categories.items():
    decomposition_rev = sm.tsa.seasonal_decompose(dft[dft["category"]==c]["total"].resample('MS').sum(), model='additive')
    trend_rev = decomposition_rev.observed
    trend_rev.plot(ax=axis, title=f"{c}",color="lightgray")
    if (c == "Event") or (c == "Pro Shop"):
        axis.get_xaxis().set_visible(True)
        axis.get_yaxis().set_visible(False)
        axis.grid(False)
    else:
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        
plt.subplots_adjust(hspace=0.4)
# plt.savefig('graph/trend_obs.png', transparent=True)

What: **Trend Decomposition**
 - When we remove the residual and seasonality, these are the trends per product category. In the above chart, we see smooth trend movements.
  
So What: **Some categories are in an uptrend, others are in downtrend**
 - Uptrend
     - Admin
     - Bar
     - Tennis
     - Event
 - Downtrend
     - Fitness
     - Pro Shop
 - Discernible changepoints happened towards the latter part of 2016 that caused `Tennis` and `Event` to dip but a trend reversal shortly followed.
 - `Bar` sales continue to rise especially when Naturally opened in the middle of 2018.
 - `Fitness` sales peaked in in the middle of 2017 but in a rapid fall thereon.
 - `Pro Shop` has not seen any growth since 2016.

### 4. Decomposing Seasonality

#### Decomposition of Seasonality

In [ ]:
fig = plt.figure(figsize=(16,7))

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

categories = {"Admin":ax1,"Bar":ax2,"Tennis":ax3,"Fitness":ax4,"Event":ax5,"Pro Shop":ax6}

for c,axis in categories.items():
    decomposition = sm.tsa.seasonal_decompose(dft[dft["category"]==c]["total"].resample('MS').sum(), model='additive')    
    seasons = decomposition.seasonal
    seasons.plot(ax=axis, title=f"{c}",color="#328B9B")
    if (c == "Event") or (c == "Pro Shop"):
        axis.get_yaxis().set_visible(False)
        axis.grid(True)
    else:
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        axis.grid(True)
        
fig.subplots_adjust(hspace=0.4)
plt.savefig('graph/seasonality_rev.png', transparent=True)

In [ ]:
fig = plt.figure(figsize=(16,7))

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

categories = {"Admin":ax1,"Bar":ax2,"Tennis":ax3,"Fitness":ax4,"Event":ax5,"Pro Shop":ax6}

for c,axis in categories.items():
    decomposition = sm.tsa.seasonal_decompose(dft[dft["category"]==c]["trans_id"].resample('MS').count(), model='additive')    
    seasons = decomposition.seasonal
    seasons.plot(ax=axis, title=f"{c}",color="#7FB13F")
    if (c == "Event") or (c == "Pro Shop"):
        axis.get_yaxis().set_visible(False)
        axis.grid(True)
    else:
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        axis.grid(True)
        
fig.subplots_adjust(hspace=0.4)
# plt.savefig('graph/seasonality_eng.png', transparent=True)

#### Auto Correlation and Seasonality

In [ ]:
admin_monthly = dft[dft.category=="Admin"].resample("MS")[["total"]].sum()
bar_monthly = dft[dft.category=="Bar"].resample("MS")[["total"]].sum()
tennis_monthly = dft[dft.category=="Tennis"].resample("MS")[["total"]].sum()
fitness_monthly = dft[dft.category=="Fitness"].resample("MS")[["total"]].sum()
event_monthly = dft[dft.category=="Event"].resample("MS")[["total"]].sum()
ps_monthly = dft[dft.category=="Pro Shop"].resample("MS")[["total"]].sum()

In [ ]:
acf_dfs = [admin_monthly,bar_monthly,tennis_monthly,fitness_monthly,event_monthly,ps_monthly]
pc = ["Admin","Bar","Tennis","Fitness","Event","Pro Shop"]

In [ ]:
# from statsmodels.tsa.stattools import acf, pacf
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# fig, axs = plt.subplots(6,1,figsize=(16,25), dpi= 100)

# for index,subset in enumerate(acf_dfs):
#     plot_acf(subset["total"].tolist(), lags=54, ax=axs[index])
#     axs[index].spines['top'].set_visible(False)
#     axs[index].spines['right'].set_visible(False)
#     axs[index].spines['bottom'].set_visible(False)
#     axs[index].spines['left'].set_visible(False)

# for index,cat in enumerate(pc):
#     axs[index].set_title(cat)


# plt.subplots_adjust(hspace=0.4)

In [ ]:
sales_hourly = dft.pivot_table(index=dft.index.strftime("%D"), columns="category", values="total",aggfunc=np.mean,fill_value=0)
engagement_hourly = dft.pivot_table(index=dft.index.strftime("%D"), columns="category", values="total",aggfunc="count",fill_value=0)

In [ ]:
fig = plt.figure(figsize=(16,16))
plt.suptitle("Average Sales By Hour of Day")

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

categories = {"Admin":ax1,"Bar":ax2,"Tennis":ax3,"Fitness":ax4,"Event":ax5,"Pro Shop":ax6}

for c,axis in categories.items():
    sales_hourly[c].plot.bar(ax=axis,title=f"{c}")
    
# plt.subplots_adjust(hspace=0.4)

In [ ]:
fig = plt.figure(figsize=(16,16))
plt.suptitle("Average Engagement By Day")

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

categories = {"Admin":ax1,"Bar":ax2,"Tennis":ax3,"Fitness":ax4,"Event":ax5,"Pro Shop":ax6}

for c,axis in categories.items():
    engagement_hourly[c].plot.bar(ax=axis,title=f"{c}")
    
# plt.subplots_adjust(hspace=0.4)

### Month-per-month Predictability (Autocorrelation Function)
- Admin, Bar - the revenue of present month is very strongly correlated to the month prior at 82%, 88%
- Pro Shop - the revenue of present month is moderately correlated to the month prior at 53%
- Fitness - the revenue of present month is weakly correlated to 3 months prior at 33%
- Tennis, Events - if you want to know what tennis and events revenue will be in the present month, the biggest indication is from 6 months and 11 months ago, respectively. But these relationships are very weak.

In [ ]:
for number in range(12):
    print(f"lag month of: {number+1}")
    for df in acf_dfs:
        print(df["total"].autocorr(lag=number+1))
    print(" ")

# Modeling

In [ ]:
d = dft.copy()
d = d.rename(columns={"total":"y"})
d.index = d.index.rename("ds")

In [ ]:
d.head()

In [ ]:
dfbar = d[d.category == "Bar"].resample("D").sum()[["y"]].reset_index()
dfps = d[d.category == "Pro Shop"].resample("D").sum()[["y"]].reset_index()
dftennis = d[d.category == "Tennis"].resample("D").sum()[["y"]].reset_index()
dfadmin = d[d.category == "Admin"].resample("D").sum()[["y"]].reset_index()
dffitness = d[d.category == "Fitness"].resample("D").sum()[["y"]].reset_index()
dfevent = d[d.category == "Event"].resample("D").sum()[["y"]].reset_index()

In [ ]:
dfbar